## KnowEnG-Research/Data_Cleanup_Pipeline Feb 27, 2017 ( GeneSet Characterization )
* new improved data from aws - latest

In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd

import knpackage.toolbox as kn

sys.path.insert(1, '../../KnowEnG_Pipelines_Library/knpackage')
import redis_utilities as redisutil

sys.path.insert(1, '../../Data_Cleanup_Pipeline/src/')
import data_cleanup_toolbox as dc

sys.path.insert(1, '../src')
import dcp_test

In [2]:
tools_base_dir, no_more_nothing = os.path.split(os.getcwd())
tools_base_dir

'/Users/mojo/keg_tmp/keg_test_tools'

In [3]:
def get_geneset_characterization_run_parameters(yml_dir, yml_file_name):
    """ run_parameters = get_samples_clustering_run_parameters(yml_dir, yml_file_name) """
    run_parameters = kn.get_run_parameters(yml_dir, yml_file_name)
    run_parameters['results_directory'] = kn.create_dir(os.getcwd(), 'run_dir')
    run_parameters['pipeline_type'] = 'geneset_characterization'
    run_parameters['redis_credential']['host'] = 'knowhub.org'
    run_parameters['redis_credential']['port'] = 6379
    run_parameters['redis_credential']['password'] = 'KnowEnG'
    
    return run_parameters

In [4]:
def get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir, pheno_data_dir=None):
    """ test_result_df = get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir, pheno_data_dir) """
    
    col_list = ['phenotype_file','validation_flag','message','spreadsheet_rows','spreadsheet_cols','cleanup_time']
        
    spreadsheet_file_list = sorted(os.listdir(spreadsheet_data_dir))
    
    test_result_df = pd.DataFrame(data=np.zeros((len(spreadsheet_file_list), len(col_list))),
                                  index=spreadsheet_file_list, columns=col_list)
    if pheno_data_dir is not None:
        pheno_file_list = sorted(os.listdir(pheno_data_dir))
        for pheno_file in pheno_file_list:
            spreadsheet_list = get_spreadsheets_for_pheno(pheno_file, spreadsheet_file_list)
            for spreadsheet_file in spreadsheet_list:
                test_result_df.loc[spreadsheet_file, 'phenotype_file'] = pheno_file
        
    return test_result_df

In [5]:
def test_geneset_characterization_cleanup(run_parameters):
    """ test_result_df = test_samples_clustering_cleanup(run_parameters) """
    
    spreadsheet_data_dir = run_parameters['spreadsheet_data_dir']
    
    test_result_df = get_spreadsheet_phenotype_dataframe(spreadsheet_data_dir)
    
    for spreadsheet_file in list(test_result_df.index):
        print(spreadsheet_file)
        run_parameters['spreadsheet_name_full_path'] = os.path.join(spreadsheet_data_dir, spreadsheet_file)

        tt = 0.0
        validation_flag = False
        message = "Failed to finish"

        try:
            t0 = time.time()
            validation_flag, message = dc.run_geneset_characterization_pipeline(run_parameters)
            tt = time.time() - t0
        except:
            pass

        test_result_df.loc[spreadsheet_file, 'message'] = message
        test_result_df.loc[spreadsheet_file, 'cleanup_time'] = tt
        test_result_df.loc[spreadsheet_file, 'validation_flag'] = validation_flag
            
    return test_result_df

In [6]:
yml_file_name = 'dcp_test_tmplate.yml'
yml_dir = os.path.join(tools_base_dir, 'data/run_files')
run_parameters = get_geneset_characterization_run_parameters(yml_dir, yml_file_name)

run_parameters['spreadsheet_data_dir'] = '/Users/lanier4/pipeline_spreadsheets/aws_v3/GSC_G_spreadsheets'

for k in sorted(run_parameters.keys()):
    print(k,':\t',run_parameters[k])

pheno_data_dir :	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets
phenotype_full_path :	 ../data/spreadsheets/TEST_1_phenotype.tsv
pipeline_type :	 geneset_characterization
redis_credential :	 {'password': 'KnowEnG', 'port': 6379, 'host': 'knowhub.org'}
results_directory :	 /Users/lanier4/dlanier_KnowEnG/keg_test_tools/notebooks/run_dir1488395011429015
run_directory :	 /Users/lanier4/dlanier_KnowEnG/keg_test_tools/data/run_files
run_file :	 dcp_test_tmplate.yml
source_hint :	 
spreadsheet_data_dir :	 /Users/lanier4/pipeline_spreadsheets/aws_v3/GSC_G_spreadsheets
spreadsheet_name_full_path :	 ../data/spreadsheets/TEST_1_gene_expression.tsv
taxonid :	 9606


In [7]:
test_result_df = test_geneset_characterization_cleanup(run_parameters)

Dmel.bdgp.G.insitu_domain.binary.a.df
Hsap.ccle.G.gene_mut.binary.a.df
Hsap.dream11.G.exome_mut.binary.a.df
Hsap.dream7_c1.G.rnaseq_expressed.binary.a.df
Hsap.nbs_LUAD.G.gene_som_mut.binary.a.df
Hsap.nbs_OV.G.gene_som_mut.binary.a.df
Hsap.nbs_UCEC.G.gene_som_mut.binary.a.df
Hsap.tcga_ucsc.G.som_mut.binary.a.df
Mmus.aggression.G.diff_expr.binary.a.df


In [9]:
run_cleanup = False
test_result_df = dcp_test.test_data_cleanup(run_parameters, run_cleanup)


	Start testing geneset_characterization at Wed Mar  1 15:12:36 2017
	 Dmel.bdgp.G.insitu_domain.binary.a.df
	 Hsap.ccle.G.gene_mut.binary.a.df
	 Hsap.dream11.G.exome_mut.binary.a.df
	 Hsap.dream7_c1.G.rnaseq_expressed.binary.a.df
	 Hsap.nbs_LUAD.G.gene_som_mut.binary.a.df
	 Hsap.nbs_OV.G.gene_som_mut.binary.a.df
	 Hsap.nbs_UCEC.G.gene_som_mut.binary.a.df
	 Hsap.tcga_ucsc.G.som_mut.binary.a.df
	 Mmus.aggression.G.diff_expr.binary.a.df
